# T2 SME0104 - Cálculo Numérico
## Recomendação de filmes com SVD

Alunos: 
- Kauê Hunnicutt Bazilli - 11212226
- Matheus Vieira Gonçalves - 11200397
- Pedro Henrique dias Junqueira de Souza - 11294312

# Explicação

O projeto desenvolvido foi um sistema de recomendação de filmes colaborativo baseado em avaliações de filmes feitas por usuários. 
Para tal, foi utilizada a [versão 'small' mais recente do dataset MovieLens](https://grouplens.org/datasets/movielens/latest/), contendo aproximadamente 100836 avaliações de 9724 filmes, feitas por 610 usuários. As avaliações foram coletadas do site MovieLens. O dataset inclui outros dados que não foram utilizados.

O objetivo do sistema desenvolvido é recomendar filmes a usuários similares com base nas avaliações gerais no sistema e nas avaliações feitas pelo usuário que recebe as recomendações.

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

In [2]:
# Inicialização e contagem dos dados
data = pd.read_csv('./dataset/ratings.csv')

user_count = len(data['userId'].unique() )
movie_count = len(data['movieId'].unique())
rating_count = len(data)

print(f"Contagem de usuários: {user_count}")
print(f"Contagem de filmes: {movie_count}")
print(f"Contagem de avaliações: {rating_count}\n")

# Mapeando os títulos dos filmes usando os IDs dos filmes como chave
movie_data = pd.read_csv('./dataset/movies.csv')

movie_titles = {}

for index, row in movie_data.iterrows():
    movie_id = row['movieId']
    movie_title = row['title']
    movie_titles[movie_id] = movie_title

Contagem de usuários: 610
Contagem de filmes: 9724
Contagem de avaliações: 100836



In [3]:
# Criação de uma matriz com usuários como linhas e filmes como colunas,
# e as entradas em [i][j] como a nota que o usuário representado na linha i
# deu a um filme na coluna j

# Associação de uma linha a cada usuário
# Ex: "Usuário 15 está na linha 14"
users = data['userId'].unique().astype('int')
user_rows = {}
count = 0

for user in users:
    user_rows[user] = count
    count += 1

row_users = {v: k for k, v in user_rows.items()}

# Associação de uma coluna a cada filme
# Ex: "Filme 22 está na linha 21"
movies = data['movieId'].unique().astype('int')
movie_columns = {}
count = 0

for movie in movies:
    movie_columns[movie] = count
    count += 1

column_movies = {v: k for k, v in movie_columns.items()}

# Definição da matriz estudada e inserção dos valores
ratings_mat = np.zeros(shape=(len(users),len(movies)))

for index, row in data.iterrows():
    user = row['userId'].astype('int')
    movie = row['movieId'].astype('int')
    rating = row['rating'].astype('float')

    ratings_mat[user_rows[user]][movie_columns[movie]] = rating

print(f"Usuário 1 deu nota {ratings_mat[user_rows[2]][movie_columns[1]]} para o filme {movie_titles[1]}")


Usuário 1 deu nota 0.0 para o filme Toy Story (1995)


In [67]:
u, sigma, v = np.linalg.svd(ratings_mat)

In [82]:
def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

def get_recommendation(mat, user, count):
    # Dicionário de semelhança de cossenos
    similarities = {} 
    for col in range(0, mat.shape[1]):
        sim = cosine_similarity(mat[:,user], mat[:,col])
        similarities[col] = sim
    
    # sort dictionary based on similarities 
    sim_dct = {k: v for k, v in sorted(similarities.items(), key=lambda item: item[1], reverse = True)}

    similar_ids = list(sim_dct.keys())[1:count+1]
    return similar_ids


print(get_recommendation(u,4,10))

[1, 25, 26, 1609, 320, 1027, 34, 12, 575, 111]


In [29]:
# Criação de um usuário hipotético para testes
# O usuário criado gosta de filmes de Drama, mas normalmente não gosta
# se o filme tiver elementos de comédia.
import random

movie_count = len(movie_data['movieId'].unique())
user_id = 611
user_ratings = []

# Número máximo de avaliações possíveis. 
# O número de avaliações gerado vai ser em torno de 1/5 desse valor.
# (O algoritmo apenas seleciona filmes de drama, e desses, nem todos são escolhidos)
max_ratings = 1000 

for index, row in movie_data.iterrows():
    movie_id = row['movieId']
    movie_title = row['title']
    movie_genres = row['genres']

    # Apenas avaliações de filmes com a tag "Drama"
    if 'Drama' in movie_genres:
        p = max_ratings/movie_count
       
        if len(movie_genres) == 5: # O filme tem apenas a tag "Drama"
            p *= 1
        elif 'Comedy' in movie_genres: # O filme tem a tag "Comédia"
            p *= 0.005
        else:
            p *= 0.5

        if(random.random() < p):
            user_ratings.append({})
            user_ratings[-1]["userId"] = user_id
            user_ratings[-1]["movieId"] = movie_id
            user_ratings[-1]["timestamp"] = 0 #dado indiferente

            rating = '0.0' #A nota dada

            if len(movie_genres) == 5:
                rating = random.choice(['4.0','4.5','5.0'])
            elif 'Comedy' in movie_genres: 
                rating = random.choice(['0.5','1.0','1.5','2.0'])
            else:
                rating = random.choice(['3.0','3.5','4.0','4.5','5.0'])
            
            user_ratings[-1]["rating"] = rating

print(f"Número de avaliações geradas: {len(user_ratings)}")

# Imprime os dados como CSV, pronto para ser colado no dataset
for rating in user_ratings:
    print(f"{rating['userId']},{rating['movieId']},{rating['rating']},{rating['timestamp']}")

Número de avaliações geradas: 220
611,14,4.5,0
611,40,4.5,0
611,43,5.0,0
611,57,4.0,0
611,80,5.0,0
611,100,5.0,0
611,111,5.0,0
611,117,4.0,0
611,161,3.0,0
611,184,4.0,0
611,213,5.0,0
611,365,4.5,0
611,389,3.0,0
611,408,5.0,0
611,427,4.5,0
611,495,5.0,0
611,580,4.5,0
611,636,4.0,0
611,650,4.5,0
611,773,5.0,0
611,786,4.5,0
611,875,3.5,0
611,981,4.0,0
611,1051,4.5,0
611,1116,4.0,0
611,1163,4.5,0
611,1183,3.0,0
611,1185,4.0,0
611,1337,3.5,0
611,1351,3.5,0
611,1399,4.5,0
611,1572,4.0,0
611,1589,3.0,0
611,1597,3.5,0
611,1624,4.5,0
611,1678,4.0,0
611,1694,4.5,0
611,1757,4.5,0
611,1897,5.0,0
611,1928,4.0,0
611,1942,5.0,0
611,2314,4.0,0
611,2400,3.0,0
611,2429,5.0,0
611,2453,5.0,0
611,2544,5.0,0
611,2609,4.0,0
611,2639,5.0,0
611,2691,4.0,0
611,2744,4.5,0
611,2762,3.5,0
611,2839,4.0,0
611,2942,4.0,0
611,2976,4.5,0
611,3083,4.0,0
611,3091,4.5,0
611,3111,4.5,0
611,3135,4.0,0
611,3141,5.0,0
611,3192,4.5,0
611,3223,4.5,0
611,3423,4.5,0
611,3424,5.0,0
611,3447,4.0,0
611,3494,3.0,0
611,3556,3.0,0
611,